# Importa dependências

In [1]:
from numpy import arange, abs, array, pi, set_printoptions, diff
#from numpy import genfromtxt, zeros_like
#from scipy.optimize import minimize, differential_evolution
from sys import path as sph
from os.path import join, abspath, dirname
from pandas import read_excel
#import time
#import pickle
#import matplotlib.pyplot as plt
import importlib

dir = dirname(r'c:\\Users\\Murilo\\OneDrive\\NEMOG Murilo\\00 Circuito DTG\\3. pyPB\\cases\\murilo\\teste1_calculate.py')
sph.append(abspath(join(dir, "..\\..\\..\\2. APP/")))
sph.append(abspath(join(dir, "..\\..")))
from pbe.app.dtg_class import DTGSolution, Import_flow_DSD2, DTG_experiment, get_location

set_printoptions(precision=4)

# Importa os testes

In [2]:
pasta = r"OneDrive/NEMOG Murilo/00 Circuito DTG/6. Compilado/LP_PB"
experiments = Import_flow_DSD2(get_location(pasta), teste={88:{1,2}})

In [3]:
experiments.select_DTG(X=["E_ANM", "E_FlowLine"])
experiments.select_DTG(X=["E_ANM"])
# Como obter apenas uma DTG:
# ID vem de:
# experiments.compares['E_ANM'][0] ou [1], 0: antes, [1]: depois
# ID = experiments.compares['E_ANM'][0] ou experiments.compares['E_ANM'][1]
# experiments.get_DTG(teste=88, ID=2)

In [4]:
experiments.get_DTG(teste=88, marco=1, ID=3)

,N_escoam,Marco,tipo,ID,local,tempo,We,dinf,dsup,freq_v,freq_n,acum_v,GV_ANM,GV_choke,d_med
168,88,1.0,extrator,3,3,NaN,6.04,0.050,0.056,0.0,0.0,0.0,100.0,100.0,0.0530
169,88,1.0,extrator,3,3,NaN,6.04,0.056,0.063,0.0,0.0,0.0,100.0,100.0,0.0595
170,88,1.0,extrator,3,3,NaN,6.04,0.063,0.072,0.0,0.0,0.0,100.0,100.0,0.0675
171,88,1.0,extrator,3,3,NaN,6.04,0.072,0.081,0.0,0.0,0.0,100.0,100.0,0.0765
172,88,1.0,extrator,3,3,NaN,6.04,0.081,0.092,0.0,0.0,0.0,100.0,100.0,0.0865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,88,1.0,extrator,3,3,NaN,6.04,812.000,918.100,0.0,0.0,100.0,100.0,100.0,865.0500
248,88,1.0,extrator,3,3,NaN,6.04,918.100,1037.000,0.0,0.0,100.0,100.0,100.0,977.5500
249,88,1.0,extrator,3,3,NaN,6.04,1037.000,1173.000,0.0,0.0,100.0,100.0,100.0,1105.0000
250,88,1.0,extrator,3,3,NaN,6.04,1173.000,1326.000,0.0,0.0,100.0,100.0,100.0,1249.5000


# Calcula ou importa a perda de carga e propriedades

In [5]:
experiments.calc_DP_GV()
experiments.get_prop(dir)

# Define todos os dados experimentais em um objeto só

In [6]:
experiments.preparaDados()

In [7]:
experiments.dados

,N_emul,N_escoam,C_agua [%],Marco,ANM,TIT-02[°C],Densidade[kg/m³],FT-01[kg/min],DP[bar],Saline Water [Pa.s],Saline Water [kg/m³],AW68 [Pa.s],AW68 [kg/m³],W/O [Pa.s],W/O [kg/m³],sigma [N/m]
0,48,88,1,1,100,21.496962,874.041658,29.982250,0.798624,0.001070,1022.020652,0.150081,871.835006,0.138214,873.336481,0.0045
1,48,88,1,2,60,21.605671,874.189594,29.998394,0.805462,0.001068,1021.990249,0.149517,871.766778,0.137710,873.268900,0.0045


# Define a malha do balanço populacional

In [8]:
# Obtem as classe do Bettersizer e define a malha
# Create mesh
x = read_excel(dir + "/classes.xlsx")
# diameter is in micrometer and volume is in mm³
d, v = x["d"].to_numpy(), x["v [mm³]"].to_numpy()
dxi = diff(v)
xi = v[:-1] + diff(v) / 2
# Remover classes zeros
sl = slice(8, -12)
dxi, xi = dxi[sl], xi[sl]
M = len(xi)

experiments.reduce_DTG(M, sl)

# Define a função de simulação

In [ ]:
def teste1_solve(M, C, exp, data):
    mp = array(C)
    mp[3] *= 1e13
    pbe_solutions = DTGSolution(
        M=M,
        time=t,
        exp=exp,
        data=data,
        model_parameters=mp,
    )

    return pbe_solutions


C0 = [0.3372, 0.1098, 1.3237, 0.262]
t = arange(0.0, 10.0, 0.001)

# Roda a simulação

In [ ]:
teste1_solve(M, C0, experiments.dados.iloc[0], experiments)

In [27]:
experiments.dados.iloc[0]

N_emul                    48.000000
N_escoam                  88.000000
C_agua [%]                 1.000000
Marco                      1.000000
ANM                      100.000000
TIT-02[°C]                21.496962
Densidade[kg/m³]         874.041658
FT-01[kg/min]             29.982250
DP[bar]                    0.798624
Saline Water [Pa.s]        0.001070
Saline Water [kg/m³]    1022.020652
AW68 [Pa.s]                0.150081
AW68 [kg/m³]             871.835006
W/O [Pa.s]                 0.138214
W/O [kg/m³]              873.336481
sigma [N/m]                0.004500
Name: 0, dtype: float64

'AW68 [kg/m³]'